# Practice Session 09: Data streams

In this session we will take a large corpus of queries and compute statistics on them using methods for data stream sampling.

# 0. Preliminaries

The dataset we will use contains the most 1,000 prolific users from the AOL Query Log (2006), a dataset released for research, and later retracted [[download link](https://github.com/wasiahmad/aol_query_log_analysis)]. The idea of this practice is to obtain some statistics on this file **without** storing parts of the file in main memory.

## 0.1. Required imports

In [1]:
import io
import csv
import math
import random
import statistics

## 0.2. How to iterate through this file

In [2]:
INPUT_FILE = "user_queries.csv"
with io.open(INPUT_FILE) as file:
    reader = csv.reader(file, delimiter="\t")
    for timestamp, userid, query in reader:
        # Prints 0.1% of lines
        if random.random() < 0.001:
            print("On %s user %s issued query %s" % (timestamp, userid, query)) 

On 2006-03-01 05:27:42 user u6754960 issued query princess birthday party
On 2006-03-01 12:09:42 user u1924115 issued query wood assassinated
On 2006-03-01 16:09:16 user u4630988 issued query pygmy goat for sale maryland virginia
On 2006-03-01 18:21:09 user u9427813 issued query mainecare
On 2006-03-02 02:59:24 user u4005216 issued query poster prudence and the pill
On 2006-03-02 15:49:13 user u2648027 issued query unique rough paper developing for digital portraits
On 2006-03-02 21:10:30 user u5402023 issued query american society of appraisers
On 2006-03-02 21:38:23 user u3747913 issued query wwe toys 2006
On 2006-03-03 13:45:24 user u8920039 issued query xpeeps
On 2006-03-04 15:30:16 user u1779655 issued query ski market
On 2006-03-05 08:39:12 user u3955016 issued query simmons realty oregon
On 2006-03-05 09:07:54 user u9952934 issued query does blue select cover tubal reversal
On 2006-03-05 16:07:09 user u8193332 issued query ugg
On 2006-03-05 19:09:19 user u7536332 issued query ce

On 2006-04-20 10:46:43 user u8221292 issued query infotrac
On 2006-04-20 17:32:01 user u4483136 issued query medical malpractice cholecystectomy
On 2006-04-21 11:18:54 user u173056 issued query colorado springs north koa
On 2006-04-21 19:18:11 user u4960410 issued query back pain baltimore
On 2006-04-21 19:27:21 user u7341676 issued query acts 17-11
On 2006-04-22 09:48:10 user u1272774 issued query refractory period of an ecg
On 2006-04-22 19:16:36 user u5271275 issued query panic at the disco
On 2006-04-22 21:17:24 user u6235299 issued query garage door bottom seals
On 2006-04-22 21:36:31 user u3559305 issued query annie's annuals
On 2006-04-22 21:57:55 user u9118191 issued query uscis the document we made based on the approval or registration of this case was mailed
On 2006-04-23 04:20:23 user u7050479 issued query brides from hell
On 2006-04-23 13:56:30 user u9211097 issued query master pools
On 2006-04-23 15:32:42 user u585444 issued query new york city craigslist
On 2006-04-23 18:

# 1. Determine approximately the top-5 queries

In this query log the most frequent queries are:

* "google" (1.6% of the queries)
* "ebay" (1.5%)
* "yahoo" (1.3%)
* "myspace" (1.0%)
* "craigslist" (0.5%)

Instead of loading the entire query log in main memory, we will use reservoir sampling to determine approximately the top-5 queries.

[**CODE**] Implement a function `add_reservoir(reservoir, item, max_size)` that adds an item to the reservoir, maintaining its size. If the reservoir is already of size *max_size*, a random item is selected and evicted *before* adding the item. It is important to evict an old item *before* adding the new item.

In [3]:
def add_to_reservoir(reservoir, item, max_reservoir_size):
    # YOUR CODE HERE
    assert(len(reservoir) <= max_reservoir_size)

[**CODE**] Iterate through the file using the reservoir sampling method seen in class.

In [4]:
reservoir_size = 500
reservoir = []

with io.open(INPUT_FILE) as file:
    reader = csv.reader(file, delimiter="\t")
    i = 0
    for timestamp, userid, query in reader:
        i += 1
        # YOUR CODE HERE
        
print("Number of queries seen    : %d" % i)
print("Number of queries sampled : %d" % len(reservoir) )

Number of queries seen    : 318023
Number of queries sampled : 500


[**REPORT**] List the top-5 queries found by looking at frequencies in the reservoir. Repeat this process for various sizes of the reservoir: 10, 20, 50, 100, 200, 500, 1000, 2000, 5000. For each size:

* List the top-5 queries and their estimated frequency. If you see a query C times in the reservoir, you can estimate the query appears *C x N / reservoir_size* times in the actual sample. Dividing this quantity by *N* and multiplying by 100 will give you the percentage.

[**REPORT**] Indicate what is the minimum reservoir size you had to use to at least have an overlap of 3/5 between the queries found by the approximate method and the actual top-5.

In [5]:
freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

by_freq = sorted([(float(frequency) * (float(i)/float(reservoir_size)), query) for query, frequency in freq.items()], reverse=True)[:5]
for frequency, query in by_freq:
    print("%s %.1f%%" % (query, float(frequency)*100/float(i)))


ebay 2.0%
myspace 1.8%
google 1.6%
msn 0.8%
yahoo mail 0.6%


# 2. Determine approximately the number of users

In this sample there are exactly 1000 users. We will estimate the number of distinct users without creating a hash table with users, but instead, we will use the Flajolet-Martin probabilistic counting method.

Use this function to count trailing zeroes in the binary representation of a number.

In [6]:
def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

Use this function to generate a random hash function. Note this generates a function, so you can do `hash_function = random_hash_function()` and then call `hash_function(x)` to compute the hash value of `x`.

In [7]:
def random_hash_function():
    salt = random.random()
    return lambda string: hash(string + str(salt))

[**CODE**] Perform 10 passes over the file. In each pass, create a new hash function and use it to hash userids. Keep the maximum number of trailing zeroes seen in the hash value of a userid. 

In [8]:
number_of_passes = 10

estimates = []

for i in range(number_of_passes):
    # YOUR_CODE_HERE
    estimates.append(estimate)
    print("Estimate on pass %d: %d" % (i+1, estimate))
    
print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

Estimate on pass 1: 512
Estimate on pass 2: 512
Estimate on pass 3: 256
Estimate on pass 4: 8192
Estimate on pass 5: 8192
Estimate on pass 6: 1024
Estimate on pass 7: 128
Estimate on pass 8: 4096
Estimate on pass 9: 1024
Estimate on pass 10: 1024
* Average of estimates: 2496.0
* Median  of estimates: 1024.0


[**REPORT**] Include in your report the median of estimates obtained in 3 separate runs of your algorithm; each run should do 10 passes over the file. Indicate why the median of estimates is preferable to the average of estimates.

# 3. Deliver

Deliver:

* A zip file containing your notebook (.ipynb file) with all the [**CODE**] parts implemented.
* A 2-pages PDF report including all parts of this notebook marked with "[**REPORT**]"